In [2]:
import exp_classification as exp
import data_analysis as ana

## Experiment 0: Baseline

In [2]:
exp.model_framework = 'Sklearn'
exp.seq_len = 1
exp.run_baseline()

----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 10018.107842
Equity Peak [$]                  10430.368674
Return [%]                           0.181078
Buy & Hold Return [%]               35.237457
Return (Ann.) [%]                    0.091589
Volatility (Ann.) [%]                3.894297
Sharpe Ratio                         0.023519
Sortino Ratio                        0.033055
Calmar Ratio                         0.015967
Max. Drawdown [%]                   -5.736318
Avg. Drawdown [%]                   -1.128814
Max. Drawdown Duration      539 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                                  234
Win Rate [%]                        47.008547
Best Trade [%]                      12.713971
Worst Trade [%]                     -9.

## Experiment 1: Logistic Regression

In [2]:
exp.model_framework = 'Sklearn'
exp.model_name = 'LogisticRegression'
exp.C = 10
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
accs, aucs, sims = [], [], []
for _ in range(10):
    acc, auc, sim = exp.run()
    accs.append(acc)
    aucs.append(auc)
    sims.append(sim)

Training LogisticRegression model...
(1792, 16) (1792,)
Train Acc: 0.5463, Val Acc: 0.5350
Test Acc: 0.5120, Test AUC: 0.5000
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 10934.784736
Equity Peak [$]                  10934.784736
Return [%]                           9.347847
Buy & Hold Return [%]               35.237471
Return (Ann.) [%]                     4.62583
Volatility (Ann.) [%]                3.837617
Sharpe Ratio                         1.205391
Sortino Ratio                        2.015507
Calmar Ratio                         1.312649
Max. Drawdown [%]                   -3.524041
Avg. Drawdown [%]                   -0.519756
Max. Drawdown Duration      277 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                   84
Win R

In [4]:
ana.overall_analysis(accs, aucs, sims)

[1.20539132 1.20538856 1.20538856 1.20539132 1.51197564 1.20539006
 1.20538975 1.51197815 1.51197564 1.51197597]
[2.01550679 2.01550126 2.01550126 2.01550679 2.5961716  2.01550379
 2.01550331 2.59617801 2.5961716  2.59617224]
[1.31264935 1.31264196 1.31264196 1.31264935 1.80915872 1.31264926
 1.31264824 1.80916746 1.80915872 1.80916087]
[4.62582982 4.62582193 4.62582193 4.62582982 5.70839387 4.62582731
 4.62582477 5.70840241 5.70839387 5.70839741]


(0.51285140562249,
 0.500762064678625,
 1.3280244959084624,
 2.247771663235448,
 1.5112525882000571,
 5.058854315196489)

## Experiment 2: SVM

In [2]:
exp.model_framework = 'Sklearn'
exp.model_name = 'SVM'
exp.max_iter = 10000
exp.C = 1
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
accs, aucs, sims = [], [], []
for _ in range(10):
    acc, auc, sim = exp.run()
    accs.append(acc)
    aucs.append(auc)
    sims.append(sim)

Training SVM model...
(1792, 16) (1792,)
Train Acc: 0.5480, Val Acc: 0.5350
Test Acc: 0.5321, Test AUC: 0.5194
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  10884.75187
Equity Peak [$]                   10884.75187
Return [%]                           8.847519
Buy & Hold Return [%]               35.237428
Return (Ann.) [%]                     4.38331
Volatility (Ann.) [%]                3.636549
Sharpe Ratio                         1.205349
Sortino Ratio                         1.93216
Calmar Ratio                         1.079412
Max. Drawdown [%]                   -4.060833
Avg. Drawdown [%]                   -0.432874
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   79
Win Rate [%]        

In [4]:
ana.overall_analysis(accs, aucs, sims)

(0.532128514056225,
 0.5194391074801814,
 1.2053489210620465,
 1.9321606460963898,
 1.0794111740011727,
 4.383309190387859)

In [5]:
exp.C = 10
accs, aucs, sims = [], [], []
for _ in range(10):
    acc, auc, sim = exp.run()
    accs.append(acc)
    aucs.append(auc)
    sims.append(sim)
ana.overall_analysis(accs, aucs, sims)

Training SVM model...
(1792, 16) (1792,)


/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train Acc: 0.5458, Val Acc: 0.5100
Test Acc: 0.5100, Test AUC: 0.5042
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  9879.751215
Equity Peak [$]                  10207.547817
Return [%]                          -1.202488
Buy & Hold Return [%]               35.237471
Return (Ann.) [%]                   -0.610306
Volatility (Ann.) [%]                4.091775
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -4.53722
Avg. Drawdown [%]                    -0.88894
Max. Drawdown Duration      543 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                  148
Win Rate [%]                        48.648649
Best Trade [%] 

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train Acc: 0.5458, Val Acc: 0.5100
Test Acc: 0.5100, Test AUC: 0.5042
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  9879.750465
Equity Peak [$]                  10207.547194
Return [%]                          -1.202495
Buy & Hold Return [%]               35.237428
Return (Ann.) [%]                    -0.61031
Volatility (Ann.) [%]                4.091775
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.537223
Avg. Drawdown [%]                    -0.88894
Max. Drawdown Duration      543 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                  148
Win Rate [%]                        48.648649
Best Trade [%] 

...

Train Acc: 0.5458, Val Acc: 0.5100
Test Acc: 0.5100, Test AUC: 0.5042
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  9879.749949
Equity Peak [$]                  10207.547128
Return [%]                          -1.202501
Buy & Hold Return [%]               35.237442
Return (Ann.) [%]                   -0.610312
Volatility (Ann.) [%]                4.091774
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.537226
Avg. Drawdown [%]                   -0.888941
Max. Drawdown Duration      543 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                  148
Win Rate [%]                        48.648649
Best Trade [%] 

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Train Acc: 0.5458, Val Acc: 0.5100
Test Acc: 0.5100, Test AUC: 0.5042
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                  9879.751045
Equity Peak [$]                  10207.547987
Return [%]                           -1.20249
Buy & Hold Return [%]               35.237428
Return (Ann.) [%]                   -0.610307
Volatility (Ann.) [%]                4.091774
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.537225
Avg. Drawdown [%]                   -0.888942
Max. Drawdown Duration      543 days 00:00:00
Avg. Drawdown Duration       80 days 00:00:00
# Trades                                  148
Win Rate [%]                        48.648649
Best Trade [%] 

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(0.5100401606425703, 0.5042381775029465, 0.0, 0.0, 0.0, -0.6103084782318857)

## Experiment 3: LSTM

In [2]:
exp.model_framework = 'PyTorch'
exp.model_name = 'LSTMClassifier'
exp.seq_len = 10
exp.n_epochs = 1000
exp.hidden_size = 64
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training LSTMClassifier model...
LSTMClassifier(
  (lstm): LSTM(16, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.5667, Train Acc: 0.6433,             Val Acc: 0.5026, Val AUC: 0.4955, lr: 0.001000
Epoch: 200, Train Loss: 0.3236, Train Acc: 0.7588,             Val Acc: 0.4869, Val AUC: 0.4814, lr: 0.001000
Epoch: 300, Tr

In [4]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training LSTMClassifier model...
LSTMClassifier(
  (lstm): LSTM(16, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6081, Train Acc: 0.6343,             Val Acc: 0.5079, Val AUC: 0.5000, lr: 0.001000
Epoch: 200, Train Loss: 0.4731, Train Acc: 0.7235,             Val Acc: 0.4817, Val AUC: 0.4772, lr: 0.001000
Epoch: 300, Tr

In [5]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training LSTMClassifier model...
LSTMClassifier(
  (lstm): LSTM(16, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6222, Train Acc: 0.6439,             Val Acc: 0.5079, Val AUC: 0.5000, lr: 0.001000
Epoch: 200, Train Loss: 0.4814, Train Acc: 0.7588,             Val Acc: 0.4974, Val AUC: 0.4899, lr: 0.001000
Epoch: 300, Tr

In [6]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: LSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training LSTMClassifier model...
LSTMClassifier(
  (lstm): LSTM(16, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.5921, Train Acc: 0.6467,             Val Acc: 0.5079, Val AUC: 0.5010, lr: 0.001000
Epoch: 200, Train Loss: 0.4215, Train Acc: 0.7291,             Val Acc: 0.4921, Val AUC: 0.4875, lr: 0.001000
Epoch: 300, Tr

In [12]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Experiment 4: MLP

In [4]:
# adam
exp.model_framework = 'Sklearn'
exp.model_name = 'MLP'
exp.max_iter = 1000
exp.lr = 1e-3
exp.batch_size = 32
exp.early_stopping = True

In [5]:
acc, auc, sim = exp.run()

Training MLP model...
(1792, 16) (1792,)
Train Acc: 0.5424, Val Acc: 0.5050
Test Acc: 0.5141, Test AUC: 0.5070
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 11114.967688
Equity Peak [$]                  11114.967688
Return [%]                          11.149677
Buy & Hold Return [%]               35.237457
Return (Ann.) [%]                    5.494704
Volatility (Ann.) [%]                4.693522
Sharpe Ratio                           1.1707
Sortino Ratio                        1.872632
Calmar Ratio                         1.327519
Max. Drawdown [%]                   -4.139079
Avg. Drawdown [%]                    -0.66351
Max. Drawdown Duration      225 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                   36
Win Rate [%]        

In [2]:
# sgd + invscaling
exp.model_framework = 'Sklearn'
exp.model_name = 'MLP'
exp.max_iter = 1000
exp.lr = 1e-3
exp.batch_size = 32
exp.early_stopping = True

In [3]:
acc, auc, sim = exp.run()

Training MLP model...
(1792, 16) (1792,)
Train Acc: 0.5346, Val Acc: 0.5100
Test Acc: 0.5060, Test AUC: 0.4993
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 11146.001783
Equity Peak [$]                  11146.001783
Return [%]                          11.460018
Buy & Hold Return [%]               35.237442
Return (Ann.) [%]                    5.643652
Volatility (Ann.) [%]                 4.59982
Sharpe Ratio                         1.226929
Sortino Ratio                        2.022925
Calmar Ratio                         1.903105
Max. Drawdown [%]                   -2.965496
Avg. Drawdown [%]                   -0.648573
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                   40
Win Rate [%]        

In [10]:
# sgd + adaptive
exp.model_framework = 'Sklearn'
exp.model_name = 'MLP'
exp.max_iter = 1000
exp.lr = 1e-3
exp.batch_size = 32
exp.early_stopping = True

In [11]:
acc, auc, sim = exp.run()

Training MLP model...
(1792, 16) (1792,)
Train Acc: 0.5346, Val Acc: 0.5100
Test Acc: 0.5060, Test AUC: 0.4993
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                 11146.003054
Equity Peak [$]                  11146.003054
Return [%]                          11.460031
Buy & Hold Return [%]               35.237442
Return (Ann.) [%]                    5.643658
Volatility (Ann.) [%]                4.599819
Sharpe Ratio                          1.22693
Sortino Ratio                        2.022928
Calmar Ratio                          1.90311
Max. Drawdown [%]                   -2.965493
Avg. Drawdown [%]                   -0.648572
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                   40
Win Rate [%]        

In [3]:
# adam + no technique indicators
exp.model_framework = 'Sklearn'
exp.model_name = 'MLP'
exp.max_iter = 1000
exp.lr = 1e-3
exp.batch_size = 32
exp.early_stopping = True
exp.technical_indicators = []

In [4]:
acc, auc, sim = exp.run()

Training MLP model...
(1812, 5) (1812,)
Train Acc: 0.5491, Val Acc: 0.5050
Test Acc: 0.5198, Test AUC: 0.5000
----------------------------------------
Start                     2021-06-30 00:00...
End                       2023-06-30 00:00...
Duration                    730 days 00:00:00
Exposure Time [%]                   99.603175
Equity Final [$]                 10770.765579
Equity Peak [$]                  10770.765579
Return [%]                           7.707656
Buy & Hold Return [%]               43.284942
Return (Ann.) [%]                    3.782299
Volatility (Ann.) [%]                6.410678
Sharpe Ratio                             0.59
Sortino Ratio                        0.895222
Calmar Ratio                         0.514103
Max. Drawdown [%]                   -7.357079
Avg. Drawdown [%]                   -0.972163
Max. Drawdown Duration      515 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                    1
Win Rate [%]         

## Experiment 5: AttLSTM

In [5]:
exp.model_framework = 'PyTorch'
exp.model_name = 'AttLSTMClassifier'
exp.seq_len = 10
exp.n_epochs = 2000
exp.hidden_size = 64
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [6]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 2000
Max Iterations: 1000
Model Name: AttLSTMClassifier
Model Framework: PyTorch
----------------------------------------
Training AttLSTMClassifier model...
AttLSTMClassifier(
  (Tanh): Tanh()
  (lstm): LSTM(16, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (batch_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6645, Train Acc: 0.5676,             Val Acc: 0.5079, Val AUC: 0.5016, lr: 0.001000
Epo

## Experiment 6: GRU

In [2]:
exp.model_framework = 'PyTorch'
exp.model_name = 'GRUClassifier'
exp.seq_len = 10
exp.n_epochs = 2000
exp.batch_size = 256
exp.hidden_size = 64

In [3]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 256
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 2000
Max Iterations: 1000
Model Name: GRUClassifier
Model Framework: PyTorch
----------------------------------------
Training GRUClassifier model...
GRUClassifier(
  (gru): GRU(16, 64, num_layers=2, batch_first=True, dropout=0.8)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6612, Train Acc: 0.5923,             Val Acc: 0.5288, Val AUC: 0.5311, lr: 0.001000
Epoch: 200, Train Loss: 0.6099, Train Acc: 0.6803,             Val Acc: 0.5079, Val AUC: 0.5074, lr: 0.001000
Epoch: 300, Train L

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/torch/nn/modules/rnn.py:998: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.gru(input, hx, self._flat_weights, self.bias, self.num_layers,


In [4]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 10
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 256
Input Size: 16
Hidden Size: 64
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 2000
Max Iterations: 1000
Model Name: GRUClassifier
Model Framework: PyTorch
----------------------------------------
Training GRUClassifier model...
GRUClassifier(
  (gru): GRU(16, 64, num_layers=2, batch_first=True, dropout=0.8)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
----------------------------------------
Epoch: 100, Train Loss: 0.6689, Train Acc: 0.5928,             Val Acc: 0.5183, Val AUC: 0.5223, lr: 0.001000
Epoch: 200, Train Loss: 0.6077, Train Acc: 0.6725,             Val Acc: 0.4817, Val AUC: 0.4887, lr: 0.001000
Epoch: 300, Train L

/home/paradoxtown/miniconda3/envs/times/lib/python3.8/site-packages/torch/nn/modules/rnn.py:998: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.gru(input, hx, self._flat_weights, self.bias, self.num_layers,


## Experiment 7: Random Forest

In [2]:
exp.model_framework = 'Sklearn'
exp.model_name = 'RandomForest'
exp.technical_indicators = ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']

In [3]:
y_pred = exp.run()

Tiker: AAPL
Period: None
Interval: 1d
Features: ['Open', 'Close', 'High', 'Low', 'Volume']
Technical Indicators: ['CROCP', 'OROCP', 'LROCP', 'HROCP', 'VROCP', 'MA', 'SMA', 'EMA', 'WMA', 'RSI', 'RSIROCP']
Target: Trend
Sequence Length: 1
Train Ratio: 0.7
Test Ratio: 0.2
C: 10.0
Batch Size: 512
Input Size: 16
Hidden Size: 128
Number of Layers: 2
Number of Classes: 1
Learning Rate: 0.001
Number of Epochs: 1000
Max Iterations: 1000
Model Name: RandomForest
Model Framework: Sklearn
----------------------------------------
Training RandomForest model...
(1792, 16) (1792,)
Train Acc: 0.5826, Val Acc: 0.5100
Test Acc: 0.5181, Test AUC: 0.5265
----------------------------------------
Start                     2021-07-09 00:00...
End                       2023-06-30 00:00...
Duration                    721 days 00:00:00
Exposure Time [%]                   99.598394
Equity Final [$]                   9869.36783
Equity Peak [$]                   10231.91784
Return [%]                          -1.3